In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import AdamW  # <-- Updated import
import os
import re

# Dataset paths
train_dir = 'C:/Users/HP/Desktop/lantana_images/lantana_images/train'
test_dir = 'C:/Users/HP/Desktop/lantana_images/lantana_images/test'
validation_dir = 'C:/Users/HP/Desktop/lantana_images/lantana_images/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define checkpoint path
checkpoint_dir = "./checkpoints_adamW"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint_lantana_epoch_{epoch:02d}.h5")

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze all base model layers initially
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)

# Find latest checkpoint
def get_latest_checkpoint():
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("model_checkpoint_lantana_epoch")]
    if not checkpoint_files:
        return None, 0
    epochs_trained = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in checkpoint_files]
    latest_epoch = max(epochs_trained)
    latest_checkpoint = os.path.join(checkpoint_dir, f"model_checkpoint_lantana_epoch_{latest_epoch:02d}.h5")
    return latest_checkpoint, latest_epoch

latest_checkpoint, last_trained_epoch = get_latest_checkpoint()

# Load the latest model checkpoint if available
if latest_checkpoint and os.path.exists(latest_checkpoint):
    print(f"Resuming training from checkpoint: {latest_checkpoint}")
    model = load_model(latest_checkpoint)
else:
    print("No saved model found. Starting fresh.")

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

# Training Phases
epochs_phase_1 = 10
epochs_phase_2 = 5
epochs_phase_3 = 5

# Phase 1: Train top layers with AdamW
if last_trained_epoch < epochs_phase_1:
    print(f"Starting Phase 1 (continuing from epoch {last_trained_epoch + 1})")
    model.compile(
        optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    history1 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1

# Phase 2: Unfreeze top 5 layers
if last_trained_epoch < epochs_phase_1 + epochs_phase_2:
    for layer in vgg19_base.layers[-5:]:
        layer.trainable = True
    for layer in densenet_base.layers[-5:]:
        layer.trainable = True

    model.compile(
        optimizer=AdamW(learning_rate=1e-5, weight_decay=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print(f"Starting Phase 2 (continuing from epoch {last_trained_epoch + 1})")
    history2 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1 + epochs_phase_2

# Phase 3: Unfreeze all layers for fine-tuning
if last_trained_epoch < epochs_phase_1 + epochs_phase_2 + epochs_phase_3:
    for layer in vgg19_base.layers:
        layer.trainable = True
    for layer in densenet_base.layers:
        layer.trainable = True

    model.compile(
        optimizer=AdamW(learning_rate=1e-6, weight_decay=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print(f"Starting Phase 3 (continuing from epoch {last_trained_epoch + 1})")
    history3 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2 + epochs_phase_3,
        callbacks=[checkpoint_callback]
    )

# Final evaluation
test_loss, test_accuracy = model.evaluate(test_generator)
val_loss, val_accuracy = model.evaluate(validation_generator)

print(f'Test Loss (Hibiscus): {test_loss}')
print(f'Test Accuracy (Hibiscus): {test_accuracy}')
print(f'Validation Loss (Hibiscus): {val_loss}')
print(f'Validation Accuracy (Hibiscus): {val_accuracy}')



Found 10872 images belonging to 2 classes.
Found 2330 images belonging to 2 classes.
Found 2330 images belonging to 2 classes.
No saved model found. Starting fresh.
Starting Phase 1 (continuing from epoch 1)
Epoch 1/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8868 - loss: 0.2823
Epoch 1: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_01.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 1035s 3s/step - accuracy: 0.8870 - loss: 0.2820 - val_accuracy: 0.9727 - val_loss: 0.0938
Epoch 2/10
  1/339 ━━━━━━━━━━━━━━━━━━━━ 14:46 3s/step - accuracy: 1.0000 - loss: 0.0932
Epoch 2: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_02.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 202s 589ms/step - accuracy: 1.0000 - loss: 0.0932 - val_accuracy: 0.9722 - val_loss: 0.0910
Epoch 3/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9762 - loss: 0.0797
Epoch 3: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_03.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 1140s 3s/step - accuracy: 0.9762 - loss: 0.0797 - val_accuracy: 0.9831 - val_loss: 0.0651
Epoch 4/10
  1/339 ━━━━━━━━━━━━━━━━━━━━ 15:15 3s/step - accuracy: 1.0000 - loss: 0.0220
Epoch 4: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_04.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 203s 592ms/step - accuracy: 1.0000 - loss: 0.0220 - val_accuracy: 0.9835 - val_loss: 0.0618
Epoch 5/10
339/339 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9844 - loss: 0.0529
Epoch 5: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_05.h5


...

339/339 ━━━━━━━━━━━━━━━━━━━━ 3530s 10s/step - accuracy: 0.9537 - loss: 0.1286 - val_accuracy: 0.9774 - val_loss: 0.0668
Epoch 17/20
  1/339 ━━━━━━━━━━━━━━━━━━━━ 52:58 9s/step - accuracy: 0.9688 - loss: 0.0573
Epoch 17: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_17.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 187s 524ms/step - accuracy: 0.9688 - loss: 0.0573 - val_accuracy: 0.9761 - val_loss: 0.0695
Epoch 18/20
339/339 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9777 - loss: 0.0610
Epoch 18: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_18.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 3432s 10s/step - accuracy: 0.9777 - loss: 0.0610 - val_accuracy: 0.9822 - val_loss: 0.0450
Epoch 19/20
  1/339 ━━━━━━━━━━━━━━━━━━━━ 53:14 9s/step - accuracy: 1.0000 - loss: 0.0032
Epoch 19: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_19.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 187s 524ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 0.9835 - val_loss: 0.0398
Epoch 20/20
339/339 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9817 - loss: 0.0514
Epoch 20: saving model to ./checkpoints_adamW\model_checkpoint_lantana_epoch_20.h5


339/339 ━━━━━━━━━━━━━━━━━━━━ 3458s 10s/step - accuracy: 0.9817 - loss: 0.0514 - val_accuracy: 0.9926 - val_loss: 0.0247
73/73 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.9936 - loss: 0.0162
73/73 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9903 - loss: 0.0307
Test Loss (Hibiscus): 0.022593431174755096
Test Accuracy (Hibiscus): 0.9909871220588684
Validation Loss (Hibiscus): 0.02531866915524006
Validation Accuracy (Hibiscus): 0.9922747015953064
